In [1]:
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import requests
import time

In [2]:
def get_links(keyword="data engineer"):
    links=[]
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(options=options)
    try:
        url = f'https://www.welcometothejungle.com/fr/jobs?&query={keyword}&sortBy=mostRecent'
        driver.get(url)
    
        time.sleep(5)
        nb_page = int(driver.find_elements(By.CLASS_NAME, 'jCRLMV')[-2].text)
        for page in range(nb_page):
            items = driver.find_elements(By.CLASS_NAME, 'ais-Hits-list-item')
            
            stop_outer_loop = False  # Flag to break the outer loop
            
            for item in items:
                links.append(item.find_element(By.CLASS_NAME, 'iPeVkS').get_attribute("href"))
                try:
                    date_str = item.find_element(By.TAG_NAME, 'time').get_attribute('datetime')
                    date = datetime.fromisoformat(date_str.replace('Z', '+00:00'))

                    if datetime.now() - date > timedelta(days=7):
                        stop_outer_loop = True
                        break
                except:
                    pass
            
            if stop_outer_loop:
                break
        
        driver.find_elements(By.CSS_SELECTOR, 'a.sc-IqJVf.evElzU')[-1].click()
        time.sleep(5)
    except:
        print("Error Keyword ..") 
    
    driver.quit()
    return links


In [3]:
def get_data(keyword="data engineer"):
    List_job = []
    links = get_links(keyword)

    print(keyword, 'Job extraction ...')
    
    for i in range(len(links)):
        job = {}
        resp = requests.get(links[i])
        soup = BeautifulSoup(resp.text,'html')

        try:
            job['Job_Link'] = links[i]
        except:
            job['Job_Link'] = None

        try: 
            ul = soup.find('div',{'class':'fhzEMX'}).find_all('ul')
            job_text = ''
            for li in ul:
                job_text += ' '+li.text
            if job_text=='':
                job_text = soup.find('div',{'class':'kqgROr'}).text
            job["Job_txt"] = job_text
        except:
            job["Job_txt"] = None

        try:
            job["company"] = soup.find('span',{'class':'sc-gvZAcH lpuzVS wui-text'}).text.strip()
        except:
            job["company"] = None

        try:
            job["job_title"] = soup.find('h2', {'class':"sc-gvZAcH lodDwl wui-text"}).text.strip()
        except:
            job["job_title"] = None
        
        try:
            job_villes = soup.find_all('div',{'class':'sc-bOhtcR eDrxLt'})[1].find_all('span',{'class':'dhOyPm'})
            job_ville=''
            for ville in job_villes:
                job_ville += ville.text.strip()
            
            job["Location"] = job_ville
        except:
            job["Location"] = None

        try:
          job["job_type"] = soup.find_all('div',{'class':'sc-bOhtcR eDrxLt'})[0].text
        except:
            job["job_type"] = None

        try:
            date = soup.find('time').get('datetime')
            dt_obj = datetime.strptime(date, "%Y-%m-%dT%H:%M:%SZ")

            date = dt_obj.date().isoformat()
            job["job_date"] = date
        except:
            job["job_date"] = None
        
        
        List_job.append(job)
        if i%30==0:
            print(i,'Jobs read')

    return pd.DataFrame(List_job)

In [4]:
def get_datas(Keywords):
    data = pd.DataFrame()
    for key in Keywords:
        df = get_data(key)
        data = pd.concat([data,df], axis=0)
    print('Done')
    
    return data

In [5]:
Keywords = ['data analyst','data scientist','data engineer','web developer','mobile developer']
job_data= get_datas(Keywords)
job_data 

Error Keyword ..
data analyst Job extraction ...
Error Keyword ..
data scientist Job extraction ...
data engineer Job extraction ...
0 Jobs read
30 Jobs read
60 Jobs read
90 Jobs read
120 Jobs read
150 Jobs read
180 Jobs read
210 Jobs read
240 Jobs read
270 Jobs read
300 Jobs read
330 Jobs read
360 Jobs read
390 Jobs read
420 Jobs read
450 Jobs read
480 Jobs read
510 Jobs read
540 Jobs read
570 Jobs read
600 Jobs read
630 Jobs read
660 Jobs read
690 Jobs read
720 Jobs read
750 Jobs read
780 Jobs read
810 Jobs read
840 Jobs read
870 Jobs read
900 Jobs read
930 Jobs read
960 Jobs read
990 Jobs read
Error Keyword ..
web developer Job extraction ...
mobile developer Job extraction ...
0 Jobs read
30 Jobs read
60 Jobs read
90 Jobs read
120 Jobs read
150 Jobs read
180 Jobs read
210 Jobs read
Done


,Job_Link,Job_txt,company,job_title,Location,job_type,job_date
0,https://www.welcometothejungle.com/fr/companie...,Animer des ateliers afin d’étudier et cadrer ...,Elevate,Cloud Data Engineer Senior,Paris,CDI,2024-07-11
1,https://www.welcometothejungle.com/fr/companie...,Collaborer étroitement avec l’équipe Trésorer...,France Active,Ingénieur.e Date Finance,Montreuil,Alternance(12 à 24 mois),2024-07-11
2,https://www.welcometothejungle.com/fr/companie...,Gestion et suivi de projets requérant des com...,Converteo,Consultant Senior Analytics Engineering H/F - CDI,Paris,CDI,2024-07-11
3,https://www.welcometothejungle.com/fr/companie...,Élaborer et diriger la stratégie d’analyse de...,Lucky Cart,Data Engineer Senior,Paris,CDI,2024-07-11
4,https://www.welcometothejungle.com/fr/companie...,"Tu implémentes des modèles, processus, traite...",JAKALA,Data Engineer / Kafka Senior,Paris,CDI,2024-07-11
...,...,...,...,...,...,...,...
235,https://www.welcometothejungle.com/fr/companie...,participer aux spécifications fonctionnelles ...,LumApps,iOS Développeur (F/H) - Lyon,Tassin-la-Demi-Lune,CDI,2024-07-08
236,https://www.welcometothejungle.com/fr/companie...,3 développeurs iOS (1 Lead + 2 Junior) au sei...,Pictarine,Senior iOS Developer,Toulouse,CDI,2024-07-08
237,https://www.welcometothejungle.com/fr/companie...,Participer aux rituels d’équipes (développeme...,theTribe,Développeur·euse web fullstack Confirmé·e - Ja...,Nantes,CDI,2024-07-08
238,https://www.welcometothejungle.com/fr/companie...,None,Hivebrite,Mobile Senior Engineer (React Native),Paris,CDI,2024-07-08


In [7]:
csv_filename = "welcome_jungle.csv"
job_data.to_csv(csv_filename, index=False)
print(f"Le fichier CSV a été créé avec succès sous le nom {csv_filename}.")


Le fichier CSV a été créé avec succès sous le nom welcome_jungle.csv.


In [6]:
job_data.tail()

,Job_Link,Job_txt,company,job_title,Location,job_type,job_date
235,https://www.welcometothejungle.com/fr/companie...,participer aux spécifications fonctionnelles ...,LumApps,iOS Développeur (F/H) - Lyon,Tassin-la-Demi-Lune,CDI,2024-07-08
236,https://www.welcometothejungle.com/fr/companie...,3 développeurs iOS (1 Lead + 2 Junior) au sei...,Pictarine,Senior iOS Developer,Toulouse,CDI,2024-07-08
237,https://www.welcometothejungle.com/fr/companie...,Participer aux rituels d’équipes (développeme...,theTribe,Développeur·euse web fullstack Confirmé·e - Ja...,Nantes,CDI,2024-07-08
238,https://www.welcometothejungle.com/fr/companie...,None,Hivebrite,Mobile Senior Engineer (React Native),Paris,CDI,2024-07-08
239,https://www.welcometothejungle.com/fr/companie...,Expérience de 6 ans minimum dans le développe...,KLANIK,Développeur Android / Kotlin senior F/H,Montpellier,CDI,2024-07-08


In [41]:
travaux_en_france = job_data[job_data['Location'].str.contains('France', na=False)]
travaux_en_france.head()


,Job_Link,Job_txt,company,job_title,Location,job_type,job_date
245,https://www.welcometothejungle.com/fr/companie...,Conduire le changement dans le secteur de la ...,FBD Group,Data Analyst (F/H),Tremblay-en-France,CDI,2024-06-25
489,https://www.welcometothejungle.com/fr/companie...,Recenser et analyser les besoins utilisateurs...,Raja,Apprentie / Apprenti Data Analyst F/H,Roissy-en-France,Alternance(12 mois),2024-05-30
934,https://www.welcometothejungle.com/fr/companie...,Conduire le changement dans le secteur de la ...,FBD Group,Process Owner - Gestion de la donnée produit (...,Tremblay-En-France,CDI,2024-06-20
352,https://www.welcometothejungle.com/fr/companie...,Participer à la définitiondes cibles et des p...,Raja,Apprenti .e datamining-ciblage F/H,Roissy-en-France,Alternance(12 mois),2024-05-23
849,https://www.welcometothejungle.com/fr/companie...,Manager l’ensemble des collaborateurs techniq...,Raja,Lead developpeur Front End F/H,Roissy-en-France,CDI,2024-07-04


In [34]:
line_4500 = job_data.iloc[4498]
line_4500


Job_Link     https://www.welcometothejungle.com/fr/companie...
Job_txt      Pour renforcer le service chargé de la gestion...
company                                              Docaposte
job_title                                Ingénieur réseaux F/H
Location                                             Marseille
job_type                                                   CDI
job_date                                            2024-06-13
Name: 1019, dtype: object

In [39]:
job_data.columns



Index(['Job_Link', 'Job_txt', 'company', 'job_title', 'Location', 'job_type',
       'job_date'],
      dtype='object')